In [1]:
import numpy as np
import cv2, glob, os
from scipy import ndimage

def croptoaspect(img):
    H, W, _ = img.shape

    aspect_ratio = 11 / 8.5

    new_W = int(H * aspect_ratio)
    x0 = int((W - new_W) / 2)
    x1 = W - x0
    return img[:, x0:x1]
def getCom(img):
   #Bereich für die Roten Pixel definieren
    lower_red = np.array([100, 0, 0])
    upper_red = np.array([255, 80, 80])

    #Maske erstellen
    mask = cv2.inRange(img, lower_red, upper_red)
    com = ndimage.center_of_mass(mask)
    return com
def getR(vc):
    up = np.array([0, 1, 0], dtype=float)
    origin = np.array([0, 0, 0], dtype=float)

    # Kamera schaut zum Ursprung
    zc = (origin - vc) / np.linalg.norm(origin - vc)

    # Rechte-Hand-System aufspannen
    xc = np.cross(up, zc)
    xc /= np.linalg.norm(xc)

    yc = np.cross(zc, xc)

    # Spalten = Achsen im Welt-KS
    R = np.column_stack((-xc, -yc, zc))
    return R
def PixeltoCameraVector(com, H, W):
    """

    :param com: liste mit 2 einträgen [0]: Vertikal, [1]: Horizontal
    :param H:   integer für Bildhöhe
    :param W:   integer für Bildbreite
    :return:    np.array mit xyz
    """
    thetay = np.deg2rad(26.99)
    thetax = 2 * np.arctan(np.tan(thetay / 2) * W / H)
    #fokale längen
    fx = W / (2 * np.tan(thetax / 2))
    fy = H / (2 * np.tan(thetay / 2))
    #Bildmittelpunkte
    cx = W / 2
    cy = H / 2
    #Kameramatrix
    K = np.eye(3)
    K[0, 0] = fx
    K[1, 1] = fy
    K[0, 2] = cx
    K[1, 2] = cy
    #Schwerpunktsvektor
    sv = np.array([com[1], com[0], 1])
    vector = np.linalg.inv(K) @ (sv)
    vector_norm = vector / np.linalg.norm(vector)
    return vector_norm
def getminAbstand(p1, d1, p2, d2):
    n = np.cross(d1, d2)                # Kreuzprodukt
    zähler = abs(np.dot(p1 - p2, n))    # Skalarprodukt!
    nenner = np.linalg.norm(n)          # Norm vom Kreuzprodukt
    if nenner < 1e-12:                  # Spezialfall: fast parallel
        return np.linalg.norm(np.cross(p1 - p2, d1)) / np.linalg.norm(d1)
    return zähler / nenner
def getminAbstandPunkt(p1, d1, p2, d2):
    # am besten Richtungen normieren, damit stabiler
    d1 = d1 / np.linalg.norm(d1)
    d2 = d2 / np.linalg.norm(d2)

    r = p1 - p2
    a = np.dot(d1, d1)
    b = np.dot(d1, d2)
    c = np.dot(d2, d2)
    d = np.dot(d1, r)
    e = np.dot(d2, r)

    denom = a * c - b * b
    if abs(denom) < 1e-12:
        # Spezialfall: Richtungen (fast) parallel
        # -> Projektion von p1 auf Linie 2
        t = 0.0
        s = np.dot(d2, r) / c
    else:
        t = (b * e - c * d) / denom
        s = (a * e - b * d) / denom

    q1 = p1 + t * d1
    q2 = p2 + s * d2
    q = 0.5 * (q1 + q2)   # Mittelpunkt der kürzesten Verbindung

    return q
def rotz(p, winkel):
    R_z = np.eye(3)
    R_z[0, 0] = np.cos(np.deg2rad(winkel))
    R_z[0, 2] = np.sin(np.deg2rad(winkel))
    R_z[2, 0] = -np.sin(np.deg2rad(winkel))
    R_z[2, 2] = np.cos(np.deg2rad(winkel))
    return R_z @ p

In [2]:
import matplotlib.pyplot as plt
#Kameravektoren
VC1 = np.array([50, 53.59 , 200])
VC2 = np.array([-50, 53.59 , 200])
R1 = getR(VC1)
R2 = getR(VC2)
#Bilder einlesen
base = r"G:\Meine Ablage\Studium\Master\3. Semester\Masterprojekt\Kamera_Simulationen\Solidworks\jakob"
patterns = [os.path.join(base, "**", "*.jpg"),
            os.path.join(base, "**", "*.jpeg"),
            os.path.join(base, "**", "*.png")]
images = []
for pat in patterns:
    images.extend(glob.glob(pat, recursive=True))
scannedPoints = []
angle = 0
for i in range(0, len(images), 2):
    print(os.path.basename(images[i]), os.path.basename(images[i + 1]), "aktueler Winkel: ", angle)
    img1 = cv2.cvtColor(cv2.imread(images[i]), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(images[i + 1]), cv2.COLOR_BGR2RGB)
    img1_cropped, img2_cropped = croptoaspect(img1), croptoaspect(img2)
    com1, com2 = getCom(img1_cropped), getCom(img2_cropped)
    C1_Ray_D = R1 @ PixeltoCameraVector(com1, 1760, 2277)
    C2_Ray_D = R2 @ PixeltoCameraVector(com2, 1760, 2277)
    p = getminAbstandPunkt(VC1, C1_Ray_D, VC2, C2_Ray_D)
    p_rotated = rotz(p, angle)
    angle = angle + 5

    # Stelle sicher, dass es ein flacher 3er-Vektor ist (x, y, z)
    p = np.asarray(p_rotated).ravel()           # macht aus (3,), (1,3) oder (3,1) ein (3,)
    assert p.size == 3, "Punkt hat nicht 3 Koordinaten"
    scannedPoints.append(p_rotated)


Camera3_000.png Camera4_000.png aktueler Winkel:  0
Camera3_005.png Camera4_005.png aktueler Winkel:  5
Camera3_010.png Camera4_010.png aktueler Winkel:  10
Camera3_015.png Camera4_015.png aktueler Winkel:  15
Camera3_020.png Camera4_020.png aktueler Winkel:  20
Camera3_025.png Camera4_025.png aktueler Winkel:  25
Camera3_030.png Camera4_030.png aktueler Winkel:  30
Camera3_035.png Camera4_035.png aktueler Winkel:  35
Camera3_040.png Camera4_040.png aktueler Winkel:  40
Camera3_045.png Camera4_045.png aktueler Winkel:  45
Camera3_050.png Camera4_050.png aktueler Winkel:  50
Camera3_055.png Camera4_055.png aktueler Winkel:  55
Camera3_060.png Camera4_060.png aktueler Winkel:  60
Camera3_065.png Camera4_065.png aktueler Winkel:  65
Camera3_070.png Camera4_070.png aktueler Winkel:  70
Camera3_075.png Camera4_075.png aktueler Winkel:  75
Camera3_080.png Camera4_080.png aktueler Winkel:  80
Camera3_085.png Camera4_085.png aktueler Winkel:  85
Camera3_090.png Camera4_090.png aktueler Winkel:

In [4]:
plt.close("all")
%matplotlib notebook
scannedPoints = np.asarray(scannedPoints, dtype=float).reshape(-1, 3)

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#Plot setup
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.set_box_aspect([1,1,1])
def set_axes_equal(ax):
    '''Gleiche Skalierung für alle 3 Achsen'''
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    y_range = abs(y_limits[1] - y_limits[0])
    z_range = abs(z_limits[1] - z_limits[0])

    max_range = max([x_range, y_range, z_range]) / 2

    x_middle = np.mean(x_limits)
    y_middle = np.mean(y_limits)
    z_middle = np.mean(z_limits)

    ax.set_xlim3d([x_middle - max_range, x_middle + max_range])
    ax.set_ylim3d([y_middle - max_range, y_middle + max_range])
    ax.set_zlim3d([z_middle - max_range, z_middle + max_range])

ax.scatter(scannedPoints[:, 0], scannedPoints[:, 1], scannedPoints[:, 2], c='b', s=5)

#Ausrichtung wie in solidworks/3d kamerakonventionen
ax.view_init(elev=90, azim=-90)
# Axes labels and legend
set_axes_equal(ax)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.show()


<IPython.core.display.Javascript object>

In [81]:
print(max(scannedPoints[:,1])-min(scannedPoints[:,1]))
print(max(scannedPoints[:,2])-min(scannedPoints[:,2]))


0.15762898153393223
53.24170725461751
[18.47229352 25.10820194 19.12304775]
